<a href="https://colab.research.google.com/github/Aaditya17032002/AutoML/blob/main/Hindi_POS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **First Preprocess the corpus and create output.csv file for model**

In [ ]:
import pandas as pd
import re

# example dataset
with open('1_Tagged_Hindi_Corpus.txt', 'r') as f:
    data = f.readlines()

# create an empty dataframe to store the results
df = pd.DataFrame(columns=['hindi', 'english'])

# iterate over each sentence in the dataset
for sentence in data:
    # split the sentence using regex and extract the relevant parts
    split_sentence = re.findall(r'([\u0900-\u097F]+\|[^|]+\|)([A-Za-z]+)\..*', sentence)
    # add the split sentence to the dataframe
    for word in split_sentence:
        hindi_word = re.sub(r'[^\u0900-\u097F ]', '', word[0].split('|')[0])
        english_word = re.sub(r'[^A-Z]', '', word[1].upper())
        df = df.append({'hindi': hindi_word, 'english': english_word}, ignore_index=True)

# remove rows with missing values
df = df.dropna(subset=['hindi', 'english'])

# save dataframe to CSV file
df.to_csv('output.csv', index=False)


# **load csv in X and y and then tokenize and padd the X and then split data into train and test**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the data from CSV file
preprocessed_data = pd.read_csv('output.csv')

# Split the data into X and y
X = preprocessed_data["hindi"].values
y = preprocessed_data["english"].values

# Tokenize and pad the sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=50, padding='post', truncating='post')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# **Label encode the y or map integers to each labels**

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode string labels to integer labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

y_train_cat = to_categorical(y_train, num_classes=41)
y_test_cat = to_categorical(y_test, num_classes=41)



# **Load the model and we are using bidirectional LSTM here as one bidirectional lstm layr consists of two hidden layer one for storing previous data and one for future data and also can process in both directions**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout


# Define the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=50))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(41, activation='softmax'))  # Change activation function and output size

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_cat, epochs=50, batch_size=5, validation_data=(X_test, y_test_cat))

Epoch 1/50
38/38 [==============================] - 16s 192ms/step - loss: 2.8728 - accuracy: 0.2581 - val_loss: 2.3195 - val_accuracy: 0.1702
Epoch 2/50
38/38 [==============================] - 6s 158ms/step - loss: 2.2846 - accuracy: 0.2419 - val_loss: 2.2735 - val_accuracy: 0.2128
Epoch 3/50
38/38 [==============================] - 5s 137ms/step - loss: 2.2434 - accuracy: 0.3118 - val_loss: 2.3632 - val_accuracy: 0.1702
Epoch 4/50
38/38 [==============================] - 7s 174ms/step - loss: 2.1935 - accuracy: 0.2903 - val_loss: 2.2597 - val_accuracy: 0.2128
Epoch 5/50
38/38 [==============================] - 5s 138ms/step - loss: 2.1668 - accuracy: 0.2366 - val_loss: 2.2185 - val_accuracy: 0.1702
Epoch 6/50
38/38 [==============================] - 7s 173ms/step - loss: 1.9592 - accuracy: 0.3495 - val_loss: 2.0230 - val_accuracy: 0.3191
Epoch 7/50
38/38 [==============================] - 5s 137ms/step - loss: 1.7104 - accuracy: 0.3602 - val_loss: 1.9818 - val_accuracy: 0.3830
Epoch

# **Predict by entering input and agin preprocess the text like tokenize,padding and predict after that use argmax to convert integer probability values to its normal form or readable form**

In [ ]:
new_hindi_text = "कश्मीर में नरेंद्र अधिनियम लागू हुआ"
words = new_hindi_text.split()

predicted_labels = []
for word in words:
    word_seq = tokenizer.texts_to_sequences([word])
    word_padded = pad_sequences(word_seq, maxlen=50, padding='post', truncating='post')
    prediction = model.predict(word_padded)
    predicted_index = prediction.argmax()
    predicted_label = label_encoder.inverse_transform([predicted_index])[0]
    predicted_labels.append(predicted_label)

output = ", ".join([f"{word}: {label}" for word, label in zip(words, predicted_labels)])
print(output)


1/1 [==============================] - 0s 33ms/step
कश्मीर: NNP, में: NST, नरेंद्र: NN, अधिनियम: NN, लागू: NST, हुआ: NST
